In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

df = pd.read_csv("/non_rand_discount.csv",sep=";")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# df.describe()
df.info()

In [ ]:
df.discount

In [ ]:
# defining a function that replaces US$ for nothing in order to get rid of the non-numerical values
def remove_symbols(discount):
  return float(discount.replace('US$ ', '')) 

In [ ]:
remove_symbols("US$ 110.0")

In [ ]:
df.discount = df.discount.apply(remove_symbols)


In [ ]:
df2 = df[['sales','discount','profit','age','cust_state','tenure','sales_prediction_bins','sales_prediction']]
df2.head()

In [ ]:
# relationship between discount and sales
sns.lmplot(x="discount", y="sales", data=df)

In [ ]:
# relationship between discount and profits 
sns.lmplot(x="discount", y="profit", data=df)

In [ ]:
# relationship between age and discount
sns.lmplot(x="discount", y="age", data=df)

In [ ]:
# relationship between sales prediction and discount
sns.lmplot(x="discount", y="sales_prediction", data=df)

In [ ]:
# relationship between tenure and discount
sns.lmplot(x="discount", y="tenure", data=df)


In [ ]:
# This is not correct?
# I chose lmplot since it gives you a scatterplot with a linear regresion applied to it, which makes it easier to interpret the data
sns.lmplot(x="discount", y="sales_prediction_bins", data=df)

In [ ]:
# Interesting comparison between df2 parameters and discount
sns.pairplot(df2, hue='discount')